In [1]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
import optuna
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler

import numpy as np

data = pd.read_csv('../Data/serenlens_with_features_cleaned_filtered.csv', dtype={'item_id': str}, sep=";")
all_item_ids = data['item_id'].unique()

train_data = pd.read_csv('../train_data.csv', dtype={'item_id': str}, sep=';')
test_data = pd.read_csv('../test_data.csv', dtype={'item_id': str}, sep=';')

def load_embeddings(embedding_path, item_ids):
    embeddings = np.load(embedding_path)
    return {item_id: normalize(embedding.reshape(1, -1)).flatten() for item_id, embedding in zip(item_ids, embeddings)}


train_user_ids = train_data['user_id'].unique()
train_item_ids = train_data['item_id'].unique()
train_user_id_map = {user_id: idx for idx, user_id in enumerate(train_user_ids)}
train_item_id_map = {item_id: idx for idx, item_id in enumerate(train_item_ids)}

interaction_matrix = train_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
svd = TruncatedSVD(n_components=50, random_state=42)
user_embeddings = svd.fit_transform(interaction_matrix)
item_embeddings_mf = svd.components_.T

scaler = StandardScaler()
user_embeddings = scaler.fit_transform(user_embeddings)
item_embeddings_mf = scaler.fit_transform(item_embeddings_mf)

# description embeddings
item_embeddings_dict = load_embeddings('../embeddings/all_mpnet_base_embeddings.npy', all_item_ids)

# default embedding for missing embeddings in test
default_user_embedding = np.mean(user_embeddings, axis=0)
default_item_embedding_svd = np.mean(item_embeddings_mf, axis=0)

default_user_embedding = normalize(default_user_embedding.reshape(1, -1)).flatten() 
default_item_embedding_svd = normalize(default_item_embedding_svd.reshape(1, -1)).flatten()  

train_data_samples = []
sample_weights = []

for idx, row in train_data.iterrows():
    user_idx = train_user_id_map.get(row['user_id'])
    item_idx = train_item_id_map.get(row['item_id'])
    user_embedding = normalize(user_embeddings[user_idx].reshape(1, -1)).flatten() if user_idx is not None else default_user_embedding
    item_embedding = normalize(item_embeddings_mf[item_idx].reshape(1, -1)).flatten() if item_idx is not None else default_item_embedding_svd
    item_embedding_bert = item_embeddings_dict[row['item_id']]
    item_final = np.hstack((item_embedding_bert, item_embedding))
    combined_embedding = np.hstack((user_embedding, item_final))
    train_data_samples.append((combined_embedding, row['serendipity']))



X_train, y_train = zip(*train_data_samples)
X_train = np.array(X_train)
y_train = np.array(y_train)

undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)


test_data_samples = []
for idx, row in test_data.iterrows():
    user_idx = train_user_id_map.get(row['user_id'])
    item_idx = train_item_id_map.get(row['item_id'])
    user_embedding = normalize(user_embeddings[user_idx].reshape(1, -1)).flatten() if user_idx is not None else default_user_embedding
    item_embedding = normalize(item_embeddings_mf[item_idx].reshape(1, -1)).flatten() if item_idx is not None else default_item_embedding_svd
    item_embedding_bert = item_embeddings_dict[row['item_id']]
    item_final = np.hstack((item_embedding_bert, item_embedding))
    combined_embedding = np.hstack((user_embedding, item_final))
    test_data_samples.append((combined_embedding, row['serendipity']))

X_test, y_test = zip(*test_data_samples)
X_test = np.array(X_test)
y_test = np.array(y_test)


X_train = X_resampled.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_resampled.astype(np.float32)
y_test = y_test.astype(np.float32)
scale_pos_weight = len(y_resampled[y_resampled == 0]) / len(y_resampled[y_resampled == 1])

#optuna function
def objective(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        #'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.1, 10),
        #'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.1, 10),
        'scale_pos_weight': scale_pos_weight
    }
    
    xgb_model = xgb.XGBClassifier(**param)
    
    X_train_opt, X_valid_opt, y_train_opt, y_valid_opt = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    xgb_model.fit(X_train_opt, y_train_opt, eval_set=[(X_valid_opt, y_valid_opt)], verbose=False)
    
    y_pred = xgb_model.predict(X_valid_opt)
    
    precision = precision_score(y_valid_opt, y_pred)
    recall = recall_score(y_valid_opt, y_pred)
    
    # chose metric to maximaze, F1-score
    return 2 * (precision * recall) / (precision + recall + 1e-10)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("best hyperparameters found:")
print(study.best_params)

# use best hyperparameters
best_params = study.best_params
xgb_model = xgb.XGBClassifier(**best_params)
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)

y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Test accuracy: {accuracy}')
print(f'Test precision: {precision}')
print(f'Test recall: {recall}')

[I 2024-09-21 16:01:19,417] A new study created in memory with name: no-name-179d3079-f245-4a9c-88f2-086d2bd1ff0a
C:\Users\alunoDEI\AppData\Local\Temp\ipykernel_14884\1696022603.py:106: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\alunoDEI\AppData\Local\Temp\ipykernel_14884\1696022603.py:107: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
C:\Users\alunoDEI\AppData\Local\Temp\ipykernel_14884\1696022603.py:108: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/relea

best hyperparameters found:
{'n_estimators': 281, 'max_depth': 11, 'learning_rate': 0.017623578300550613, 'subsample': 0.952097908977495, 'colsample_bytree': 0.6616284874817565}
[0]	validation_0-logloss:0.69195
[1]	validation_0-logloss:0.69122
[2]	validation_0-logloss:0.69144
[3]	validation_0-logloss:0.69264
[4]	validation_0-logloss:0.69137
[5]	validation_0-logloss:0.69077
[6]	validation_0-logloss:0.69037
[7]	validation_0-logloss:0.69020
[8]	validation_0-logloss:0.69187
[9]	validation_0-logloss:0.69152
[10]	validation_0-logloss:0.69275
[11]	validation_0-logloss:0.69148
[12]	validation_0-logloss:0.69022
[13]	validation_0-logloss:0.69008
[14]	validation_0-logloss:0.69073
[15]	validation_0-logloss:0.69113
[16]	validation_0-logloss:0.69048
[17]	validation_0-logloss:0.68984
[18]	validation_0-logloss:0.69030
[19]	validation_0-logloss:0.68958
[20]	validation_0-logloss:0.68948
[21]	validation_0-logloss:0.68950
[22]	validation_0-logloss:0.68986
[23]	validation_0-logloss:0.68915
[24]	validation_

In [11]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1337  960]
 [  36   63]]
              precision    recall  f1-score   support

         0.0       0.97      0.58      0.73      2297
         1.0       0.06      0.64      0.11        99

    accuracy                           0.58      2396
   macro avg       0.52      0.61      0.42      2396
weighted avg       0.94      0.58      0.70      2396



In [12]:
from sklearn.metrics import precision_recall_curve, f1_score, classification_report

y_pred_prob = xgb_model.predict_proba(X_test)[:, 1]  #positive class probability

# experiment different thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)

f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)

best_threshold = thresholds[np.argmax(f1_scores)]
print(f"better thresholds for F1-score: {best_threshold}")

better thresholds for F1-score: 0.8020690083503723


In [13]:
y_pred_adjusted = (y_pred_prob >= best_threshold).astype(int)

In [14]:
print(confusion_matrix(y_test, y_pred_adjusted))
print(classification_report(y_test, y_pred_adjusted))

[[2147  150]
 [  79   20]]
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.95      2297
         1.0       0.12      0.20      0.15        99

    accuracy                           0.90      2396
   macro avg       0.54      0.57      0.55      2396
weighted avg       0.93      0.90      0.92      2396



In [15]:
xgb_model.save_model('serendipity_classifier_mpmet.json')